In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [4]:
# Download the dataset
download_elec_dataset()

# Clean dataset
elec_data = clean_elec()

from sklearn.preprocessing import StandardScaler, MinMaxScaler
Std_normalization = 1
if Std_normalization:
    scaler = StandardScaler()
    temp = scaler.fit_transform(elec_data)
    norm_means = scaler.mean_
    norm_std = scaler.scale_
else:
    scaler = MinMaxScaler()
    temp = scaler.fit_transform(elec_data)

elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

Files are ready
Data loaded..
Data aggregated by hour: (26304, 370)


In [5]:
# # prompt: write pandas to csv

# import pandas as pd

# # Save the DataFrame to a CSV file
# elec_data.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/electricity/electricity.csv', index_label='date')


In [6]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig

args = NS_TransformerConfig()
args.data_path

'electricity.csv'

In [7]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [8]:
test_data[0][0].shape

(96, 371)

In [9]:
n_users = 10
user_num_ts = int(elec_data.shape[1] / n_users)
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, elec_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, elec_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [10]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(37, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(37, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
          

# Setup WANDB

In [11]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Elec dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Elec dataset",
    # Hyperparams
    config={
        "dataset": "Elec370",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 370,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [12]:
import random
from tqdm import tqdm

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_rmse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_rmse.append(rmse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sum(total_rmse)/len(total_rmse)})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 52.40694856643677
Epoch: 2 cost time: 52.24674606323242
Epoch: 3 cost time: 52.87699007987976
Epoch: 4 cost time: 51.57536315917969
Epoch: 5 cost time: 53.146167278289795


Progress:   3%|▎         | 1/30 [04:29<2:10:04, 269.12s/it]

Epoch: 1 cost time: 52.15255522727966
Epoch: 2 cost time: 52.064481019973755
Epoch: 3 cost time: 54.12309956550598
Epoch: 4 cost time: 51.28715252876282
Epoch: 5 cost time: 52.43724203109741


Progress:   7%|▋         | 2/30 [08:58<2:05:32, 269.02s/it]

...

Progress:  93%|█████████▎| 28/30 [2:04:31<08:34, 257.14s/it]

Epoch: 1 cost time: 50.30553960800171
Epoch: 2 cost time: 49.08963465690613
Epoch: 3 cost time: 50.09939122200012
Epoch: 4 cost time: 49.09493279457092
Epoch: 5 cost time: 50.41423058509827


Progress:  97%|█████████▋| 29/30 [2:08:48<04:16, 256.88s/it]

Epoch: 1 cost time: 46.960026025772095
Epoch: 2 cost time: 49.06699848175049
Epoch: 3 cost time: 50.11863684654236
Epoch: 4 cost time: 50.48432540893555
Epoch: 5 cost time: 49.51280927658081


Progress: 100%|██████████| 30/30 [2:13:01<00:00, 266.05s/it]
